In [1]:
import os
os.getcwd()

'C:\\Users\\pdharantej\\OneDrive - ALLEGIS GROUP\\Desktop\\TEK_Training\\5. Data Analysis'

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('WindowFunctions').getOrCreate()

In [4]:
df=spark.read.parquet('./emp_data.parquet')
df.show()

+-------+--------+-------+----------+-------+-----+
| emp_no|emp_name| salary|manager_id|dept_no| comm|
+-------+--------+-------+----------+-------+-----+
|1000245| PRADEEP|5000.00|      null|    100| 0.00|
|1000258|   BLAKE|2850.00|   1000245|    300|50.00|
|1000262|   CLARK|2450.00|   1000245|    100|50.00|
|1000276|   JONES|2975.00|   1000245|    200|75.00|
|1000288|   SCOTT|3000.00|   1000276|    200| 0.00|
|1000292|    FORD|3000.00|   1000276|    200| 0.00|
|1000294|   SMITH| 800.00|   1000292|    200| 0.00|
|1000299|   ALLEN|1600.00|   1000258|    300| 0.00|
|1000310|    WARD|1250.00|   1000258|    300|50.00|
|1000312|  MARTIN|1250.00|   1000258|    300|50.00|
|1000315|  TURNER|1500.00|   1000258|    300| 0.00|
|1000326|   ADAMS|1100.00|   1000288|    200| 0.00|
|1000336|   JAMES| 950.00|   1000258|    300|50.00|
|1000346|  MILLER|1300.00|   1000262|    100| 0.00|
|1000347|   DAVID|1400.00|   1000245|    500| 0.00|
+-------+--------+-------+----------+-------+-----+



In [5]:
from pyspark.sql.window import Window
from pyspark.sql.functions import count, col

In [6]:
empdf = df.select('emp_name', 'dept_no', 'salary', count('*').over(Window.partitionBy('dept_no')).alias('Total_Emps'))
empdf.show()

+--------+-------+-------+----------+
|emp_name|dept_no| salary|Total_Emps|
+--------+-------+-------+----------+
| PRADEEP|    100|5000.00|         3|
|   CLARK|    100|2450.00|         3|
|  MILLER|    100|1300.00|         3|
|   JONES|    200|2975.00|         5|
|   SCOTT|    200|3000.00|         5|
|    FORD|    200|3000.00|         5|
|   SMITH|    200| 800.00|         5|
|   ADAMS|    200|1100.00|         5|
|   BLAKE|    300|2850.00|         6|
|   ALLEN|    300|1600.00|         6|
|    WARD|    300|1250.00|         6|
|  MARTIN|    300|1250.00|         6|
|  TURNER|    300|1500.00|         6|
|   JAMES|    300| 950.00|         6|
|   DAVID|    500|1400.00|         1|
+--------+-------+-------+----------+



In [7]:
empdf = df.select('emp_name', 'dept_no', 'salary', count('*').over(Window.partitionBy('dept_no').rowsBetween(Window.currentRow+1, Window.unboundedFollowing)).alias('Reminaing_cnt'))
empdf.show()

+--------+-------+-------+-------------+
|emp_name|dept_no| salary|Reminaing_cnt|
+--------+-------+-------+-------------+
| PRADEEP|    100|5000.00|            2|
|   CLARK|    100|2450.00|            1|
|  MILLER|    100|1300.00|            0|
|   JONES|    200|2975.00|            4|
|   SCOTT|    200|3000.00|            3|
|    FORD|    200|3000.00|            2|
|   SMITH|    200| 800.00|            1|
|   ADAMS|    200|1100.00|            0|
|   BLAKE|    300|2850.00|            5|
|   ALLEN|    300|1600.00|            4|
|    WARD|    300|1250.00|            3|
|  MARTIN|    300|1250.00|            2|
|  TURNER|    300|1500.00|            1|
|   JAMES|    300| 950.00|            0|
|   DAVID|    500|1400.00|            0|
+--------+-------+-------+-------------+



In [8]:
from pyspark.sql.functions import sum

In [9]:
empdf = df.select('emp_name', 'dept_no', 'salary', sum('salary').over(Window.partitionBy('dept_no').orderBy(col('salary').desc())).alias('cummulative_sum'))
empdf.show()

+--------+-------+-------+---------------+
|emp_name|dept_no| salary|cummulative_sum|
+--------+-------+-------+---------------+
| PRADEEP|    100|5000.00|        5000.00|
|   CLARK|    100|2450.00|        7450.00|
|  MILLER|    100|1300.00|        8750.00|
|   SCOTT|    200|3000.00|        6000.00|
|    FORD|    200|3000.00|        6000.00|
|   JONES|    200|2975.00|        8975.00|
|   ADAMS|    200|1100.00|       10075.00|
|   SMITH|    200| 800.00|       10875.00|
|   BLAKE|    300|2850.00|        2850.00|
|   ALLEN|    300|1600.00|        4450.00|
|  TURNER|    300|1500.00|        5950.00|
|    WARD|    300|1250.00|        8450.00|
|  MARTIN|    300|1250.00|        8450.00|
|   JAMES|    300| 950.00|        9400.00|
|   DAVID|    500|1400.00|        1400.00|
+--------+-------+-------+---------------+



In [10]:
# cumulative sum without partitions

empdf = df.select('emp_name', 'dept_no', 'salary', sum('salary').over(Window.orderBy(col('salary').desc(), col('emp_name').desc())).alias('true_cummulative_sum'))
empdf.show()

+--------+-------+-------+--------------------+
|emp_name|dept_no| salary|true_cummulative_sum|
+--------+-------+-------+--------------------+
| PRADEEP|    100|5000.00|             5000.00|
|   SCOTT|    200|3000.00|             8000.00|
|    FORD|    200|3000.00|            11000.00|
|   JONES|    200|2975.00|            13975.00|
|   BLAKE|    300|2850.00|            16825.00|
|   CLARK|    100|2450.00|            19275.00|
|   ALLEN|    300|1600.00|            20875.00|
|  TURNER|    300|1500.00|            22375.00|
|   DAVID|    500|1400.00|            23775.00|
|  MILLER|    100|1300.00|            25075.00|
|    WARD|    300|1250.00|            26325.00|
|  MARTIN|    300|1250.00|            27575.00|
|   ADAMS|    200|1100.00|            28675.00|
|   JAMES|    300| 950.00|            29625.00|
|   SMITH|    200| 800.00|            30425.00|
+--------+-------+-------+--------------------+



In [11]:
from pyspark.sql.functions import row_number, col

empdf_update = df.select('emp_name', 'dept_no', 'salary', row_number().over(Window.partitionBy('dept_no').orderBy('salary')).alias('row_number_Dept_salary'))
empdf_update.show()

+--------+-------+-------+----------------------+
|emp_name|dept_no| salary|row_number_Dept_salary|
+--------+-------+-------+----------------------+
|  MILLER|    100|1300.00|                     1|
|   CLARK|    100|2450.00|                     2|
| PRADEEP|    100|5000.00|                     3|
|   SMITH|    200| 800.00|                     1|
|   ADAMS|    200|1100.00|                     2|
|   JONES|    200|2975.00|                     3|
|   SCOTT|    200|3000.00|                     4|
|    FORD|    200|3000.00|                     5|
|   JAMES|    300| 950.00|                     1|
|    WARD|    300|1250.00|                     2|
|  MARTIN|    300|1250.00|                     3|
|  TURNER|    300|1500.00|                     4|
|   ALLEN|    300|1600.00|                     5|
|   BLAKE|    300|2850.00|                     6|
|   DAVID|    500|1400.00|                     1|
+--------+-------+-------+----------------------+



In [12]:
# rank of employees by salary within department

from pyspark.sql.functions import rank, col

empdf_update = df.select('emp_name', 'dept_no', 'salary', rank().over(Window.partitionBy('dept_no').orderBy('salary')).alias('rank_Dept_salary'))
empdf_update.show(truncate=False)

+--------+-------+-------+----------------+
|emp_name|dept_no|salary |rank_Dept_salary|
+--------+-------+-------+----------------+
|MILLER  |100    |1300.00|1               |
|CLARK   |100    |2450.00|2               |
|PRADEEP |100    |5000.00|3               |
|SMITH   |200    |800.00 |1               |
|ADAMS   |200    |1100.00|2               |
|JONES   |200    |2975.00|3               |
|SCOTT   |200    |3000.00|4               |
|FORD    |200    |3000.00|4               |
|JAMES   |300    |950.00 |1               |
|WARD    |300    |1250.00|2               |
|MARTIN  |300    |1250.00|2               |
|TURNER  |300    |1500.00|4               |
|ALLEN   |300    |1600.00|5               |
|BLAKE   |300    |2850.00|6               |
|DAVID   |500    |1400.00|1               |
+--------+-------+-------+----------------+



In [13]:
temp_table_name = 'emp_data_csv'
df.createOrReplaceTempView(temp_table_name)

In [14]:
# import os
# os.environ["HADOOP_HOME"] = "C:/path/to/hadoop"
# os.environ["hadoop.home.dir"] = "C:/path/to/hadoop"

In [15]:
# permanent_table_name = 'emp_data_table'
# df.write.format('parquet').saveAsTable(permanent_table_name)

In [16]:
# spark.sql('select * from emp_data_table').show()

In [17]:
df = spark.sql('select * from emp_data_csv')
df.show()

+-------+--------+-------+----------+-------+-----+
| emp_no|emp_name| salary|manager_id|dept_no| comm|
+-------+--------+-------+----------+-------+-----+
|1000245| PRADEEP|5000.00|      null|    100| 0.00|
|1000258|   BLAKE|2850.00|   1000245|    300|50.00|
|1000262|   CLARK|2450.00|   1000245|    100|50.00|
|1000276|   JONES|2975.00|   1000245|    200|75.00|
|1000288|   SCOTT|3000.00|   1000276|    200| 0.00|
|1000292|    FORD|3000.00|   1000276|    200| 0.00|
|1000294|   SMITH| 800.00|   1000292|    200| 0.00|
|1000299|   ALLEN|1600.00|   1000258|    300| 0.00|
|1000310|    WARD|1250.00|   1000258|    300|50.00|
|1000312|  MARTIN|1250.00|   1000258|    300|50.00|
|1000315|  TURNER|1500.00|   1000258|    300| 0.00|
|1000326|   ADAMS|1100.00|   1000288|    200| 0.00|
|1000336|   JAMES| 950.00|   1000258|    300|50.00|
|1000346|  MILLER|1300.00|   1000262|    100| 0.00|
|1000347|   DAVID|1400.00|   1000245|    500| 0.00|
+-------+--------+-------+----------+-------+-----+



In [18]:
from pyspark.sql.functions import col,avg,sum,min,max,count
from pyspark.sql import Window

# Aggregate Window Functions

df=df.withColumn("avg_salary",avg(col("salary")).over(Window.partitionBy("dept_no").orderBy("salary")))
print(df.show())

df=df.withColumn("max_salary",max(col("salary")).over(Window.partitionBy("dept_no")))
print(df.show())

df=df.withColumn("min_salary",min(col("salary")).over(Window.partitionBy("dept_no")))
print(df.show())

df=df.withColumn("sum_salary",sum(col("salary")).over(Window.partitionBy("dept_no").orderBy("salary")))
print(df.show())

df=df.withColumn("count_salary_units",count(col("salary")).over(Window.partitionBy("dept_no")))
print(df.show())

+-------+--------+-------+----------+-------+-----+-----------+
| emp_no|emp_name| salary|manager_id|dept_no| comm| avg_salary|
+-------+--------+-------+----------+-------+-----+-----------+
|1000346|  MILLER|1300.00|   1000262|    100| 0.00|1300.000000|
|1000262|   CLARK|2450.00|   1000245|    100|50.00|1875.000000|
|1000245| PRADEEP|5000.00|      null|    100| 0.00|2916.666667|
|1000294|   SMITH| 800.00|   1000292|    200| 0.00| 800.000000|
|1000326|   ADAMS|1100.00|   1000288|    200| 0.00| 950.000000|
|1000276|   JONES|2975.00|   1000245|    200|75.00|1625.000000|
|1000288|   SCOTT|3000.00|   1000276|    200| 0.00|2175.000000|
|1000292|    FORD|3000.00|   1000276|    200| 0.00|2175.000000|
|1000336|   JAMES| 950.00|   1000258|    300|50.00| 950.000000|
|1000310|    WARD|1250.00|   1000258|    300|50.00|1150.000000|
|1000312|  MARTIN|1250.00|   1000258|    300|50.00|1150.000000|
|1000315|  TURNER|1500.00|   1000258|    300| 0.00|1237.500000|
|1000299|   ALLEN|1600.00|   1000258|   

In [19]:
from pyspark.sql.functions import row_number, rank, dense_rank, ntile, percent_rank
from pyspark.sql import Window

df=df.withColumn('row_number', row_number().over(Window.partitionBy('dept_no').orderBy('salary')))
print(df.sho)
df=df.withColumn('rank', rank().over(Window.partitionBy('dept_no').orderBy('salary')))
df=df.withColumn('dense_rank', dense_rank().over(Window.partitionBy('dept_no').orderBy('salary')))
df=df.withColumn('percent_rank', percent_rank().over(Window.partitionBy('dept_no').orderBy('salary')))
df=df.withColumn('ntile', ntile(10).over(Window.partitionBy('dept_no').orderBy('salary')))

In [23]:
df.select("emp_no","emp_name","dept_no","salary","row_number","rank","dense_rank","percent_rank","ntile").distinct().filter(df.manager_id == "1000258").show()

+-------+--------+-------+-------+----------+----+----------+------------+-----+
| emp_no|emp_name|dept_no| salary|row_number|rank|dense_rank|percent_rank|ntile|
+-------+--------+-------+-------+----------+----+----------+------------+-----+
|1000336|   JAMES|    300| 950.00|         1|   1|         1|         0.0|    1|
|1000310|    WARD|    300|1250.00|         2|   2|         2|         0.2|    2|
|1000312|  MARTIN|    300|1250.00|         3|   2|         2|         0.2|    3|
|1000315|  TURNER|    300|1500.00|         4|   4|         3|         0.6|    4|
|1000299|   ALLEN|    300|1600.00|         5|   5|         4|         0.8|    5|
+-------+--------+-------+-------+----------+----+----------+------------+-----+



In [26]:
# Analytical Window Function

# lead, lag, cume_dist
from pyspark.sql.functions import cume_dist, lag, lead

df = df.withColumn("cume_dist",cume_dist().over(Window.partitionBy("dept_no").orderBy("salary")))
df = df.withColumn("lag",lag("salary").over(Window.partitionBy("dept_no").orderBy("salary")))
df = df.withColumn("lead",lead("salary").over(Window.partitionBy("dept_no").orderBy("salary")))

df.select("emp_no","emp_name","dept_no","salary","cume_dist","lag","lead").distinct().filter(df.manager_id == "1000258").show()

+-------+--------+-------+-------+-------------------+-------+-------+
| emp_no|emp_name|dept_no| salary|          cume_dist|    lag|   lead|
+-------+--------+-------+-------+-------------------+-------+-------+
|1000310|    WARD|    300|1250.00|                0.5| 950.00|1250.00|
|1000312|  MARTIN|    300|1250.00|                0.5|1250.00|1500.00|
|1000315|  TURNER|    300|1500.00| 0.6666666666666666|1250.00|1600.00|
|1000299|   ALLEN|    300|1600.00| 0.8333333333333334|1500.00|2850.00|
|1000336|   JAMES|    300| 950.00|0.16666666666666666|   null|1250.00|
+-------+--------+-------+-------+-------------------+-------+-------+

